In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('PRSA_data.csv')

In [5]:
# 先使用未填补的数据

pm25_data = data.loc[:, "pm2.5"].values.reshape(-1, 1)
# nan_index, _ = np.where(np.isnan(pm25_data))
non_nan_index, _ = np.where(~np.isnan(pm25_data))
X, y = data.iloc[non_nan_index, 6:], data.iloc[non_nan_index, 5]

# One hot encoder

cbwd_one_hot = dict(zip(set(X['cbwd']), range(4)))
cbwd_one_hot_inverse = dict(zip(range(4), set(X['cbwd'])))
import copy

X_cbwd = copy.deepcopy(X['cbwd'].values)
X_cbwd_new = np.asarray(X_cbwd)
for i, item in enumerate(X_cbwd):
    X_cbwd_new[i] = cbwd_one_hot[item]

X['cbwd'] = X_cbwd_new

## 1.基于AIC和BIC的特征提取

In [3]:
def aic(y_true, y_pred, p):
    n = len(y_true)
    return 2 * (p + 1) + n * np.log(np.sum((y_pred - y_true) ** 2) / n)


def bic(y_true, y_pred, p):
    n = len(y_true)
    return (p + 1) * np.log(len(y_true)) + n * np.log(np.sum((y_pred - y_true) ** 2) / n)

In [4]:
from sklearn.linear_model import LinearRegression
import copy

columns = X.columns
aic_value_all = []
selected_variable_aic = []
for i in range(len(columns)):

    aic_value = []
    alternative_var = []
    temp_var = copy.copy(selected_variable_aic)
    for var in columns:

        if var in selected_variable_aic:
            continue

        temp_var.append(var)
        X_temp = X.loc[:, temp_var]
        lr = LinearRegression().fit(X_temp, y)
        aic_value.append(aic(y, lr.predict(X_temp), len(temp_var)))
        alternative_var.append(var)

    if len(aic_value) != 0:
        aic_value_all.append(min(aic_value))
        selected_variable_aic.append(alternative_var[aic_value.index(min(aic_value))])

In [5]:
selected_variable_aic

['Ir', 'Is', 'Iws', 'cbwd', 'PRES', 'TEMP', 'DEWP']

In [6]:
bic_value_all = []
selected_variable_bic = []
for i in range(len(columns)):

    bic_value = []
    alternative_var = []
    temp_var = copy.copy(selected_variable_bic)
    for var in columns:

        if var in selected_variable_bic:
            continue

        temp_var.append(var)
        X_temp = X.loc[:, temp_var]
        lr = LinearRegression().fit(X_temp, y)
        bic_value.append(aic(y, lr.predict(X_temp), len(temp_var)))
        alternative_var.append(var)

    if len(bic_value) != 0:
        aic_value_all.append(min(bic_value))
        selected_variable_bic.append(alternative_var[bic_value.index(min(bic_value))])

In [7]:
selected_variable_bic

['Ir', 'Is', 'Iws', 'cbwd', 'PRES', 'TEMP', 'DEWP']

## 2.基于LASSO回归的特征提取

In [49]:
from sklearn.linear_model import Lasso

selected_variable_lasso=[]

for i in range(150,0,-1):
    lasso=Lasso(alpha=i).fit(X,y)
    if set(X.columns[np.abs(lasso.coef_)>1e-5])==set(selected_variable_lasso):
        continue
    selected_variable_lasso.extend([i for i in X.columns[np.abs(lasso.coef_)>1e-5] if i not in selected_variable_lasso])

In [50]:
selected_variable_lasso

['Iws', 'DEWP', 'TEMP', 'PRES', 'Ir', 'cbwd', 'Is']

In [54]:
from sklearn.metrics import r2_score

lr=LinearRegression().fit(X.iloc[:,[0,1,2,4,6]],y)
r2_score(y,lr.predict(X.iloc[:,[0,1,2,4,6]]))

0.2357746597685373